# Real-time Remote Photoplethysmography (rPPG)

## Sistem Deteksi Detak Jantung Tanpa Kontak

**Deskripsi:** Implementasi sistem rPPG yang dapat mendeteksi detak jantung secara real-time menggunakan webcam tanpa kontak fisik.

### Pipeline:
1. **Face Detection & ROI Extraction** - Deteksi wajah dan ekstraksi Region of Interest
2. **Signal Extraction** - Spatial averaging pada kanal warna (terutama Green)
3. **Signal Processing** - Detrending dan Bandpass Filtering (0.67-4.0 Hz)
4. **BPM Estimation** - Transformasi Fourier untuk mendapatkan frekuensi dominan
5. **Improvements** - POS method, skin segmentation, dan visualisasi real-time

---

## 1. Import Libraries dan Setup

In [ ]:
import cv2
import numpy as np
import mediapipe as mp
from scipy import signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks
from collections import deque
import time
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import warnings
warnings.filterwarnings('ignore')

print("✓ All libraries imported successfully!")
print(f"OpenCV version: {cv2.__version__}")
print(f"NumPy version: {np.__version__}")

## 2. Face Detection dan ROI Extraction

Menggunakan MediaPipe Face Mesh untuk deteksi wajah yang lebih akurat dan mendapatkan landmark untuk skin segmentation.

In [ ]:
class FaceDetector:
    """
    Face detector menggunakan MediaPipe Face Mesh.
    Improvement: Menggunakan face landmarks untuk ROI yang lebih spesifik (dahi, pipi).
    """
    def __init__(self):
        self.mp_face_mesh = mp.solutions.face_mesh
        self.face_mesh = self.mp_face_mesh.FaceMesh(
            max_num_faces=1,
            refine_landmarks=True,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        )
        
        # Landmark indices untuk ROI yang optimal (forehead dan cheeks)
        # Forehead region
        self.forehead_indices = [10, 338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288,
                                 397, 365, 379, 378, 400, 377, 152, 148, 176, 149, 150, 136,
                                 172, 58, 132, 93, 234, 127, 162, 21, 54, 103, 67, 109]
        
        # Left and right cheek regions
        self.left_cheek_indices = [234, 227, 137, 177, 215, 138, 135, 169, 170, 140, 171, 175]
        self.right_cheek_indices = [454, 447, 366, 401, 435, 367, 364, 394, 395, 369, 396, 400]
        
    def detect_face_roi(self, frame):
        """
        Deteksi wajah dan ekstrak ROI untuk skin area.
        Returns: ROI mask dan landmark points
        """
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.face_mesh.process(rgb_frame)
        
        if not results.multi_face_landmarks:
            return None, None
        
        h, w = frame.shape[:2]
        face_landmarks = results.multi_face_landmarks[0]
        
        # Create mask for ROI
        mask = np.zeros((h, w), dtype=np.uint8)
        
        # Extract forehead region
        forehead_points = []
        for idx in self.forehead_indices:
            landmark = face_landmarks.landmark[idx]
            x = int(landmark.x * w)
            y = int(landmark.y * h)
            forehead_points.append([x, y])
        
        if len(forehead_points) > 0:
            forehead_points = np.array(forehead_points, dtype=np.int32)
            cv2.fillConvexPoly(mask, forehead_points, 255)
        
        # Extract cheek regions
        for cheek_indices in [self.left_cheek_indices, self.right_cheek_indices]:
            cheek_points = []
            for idx in cheek_indices:
                landmark = face_landmarks.landmark[idx]
                x = int(landmark.x * w)
                y = int(landmark.y * h)
                cheek_points.append([x, y])
            
            if len(cheek_points) > 0:
                cheek_points = np.array(cheek_points, dtype=np.int32)
                cv2.fillConvexPoly(mask, cheek_points, 255)
        
        return mask, face_landmarks

# Test the face detector
detector = FaceDetector()
print("✓ Face detector initialized successfully!")

## 2.5. Adaptive ROI Selection - Quality-Based

**Improvement:** ROI dipilih berdasarkan quality metrics, bukan fixed regions.
- Evaluasi exposure level
- Check variance (blood flow visibility)
- Pilih top-K regions dengan kualitas terbaik

In [ ]:
class AdaptiveROISelector:
    """
    Adaptive ROI selection berdasarkan quality metrics.
    Improvement: Tidak semua ROI digunakan, hanya yang berkualitas baik.
    """
    
    def __init__(self):
        # Define candidate ROI regions
        self.candidate_regions = {
            "forehead": [10, 338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288,
                        397, 365, 379, 378, 400, 377, 152, 148, 176, 149, 150, 136,
                        172, 58, 132, 93, 234, 127, 162, 21, 54, 103, 67, 109],
            "left_cheek": [234, 227, 137, 177, 215, 138, 135, 169, 170, 140, 171, 175],
            "right_cheek": [454, 447, 366, 401, 435, 367, 364, 394, 395, 369, 396, 400]
        }
    
    def evaluate_roi_quality(self, frame, landmarks, region_indices):
        """
        Evaluasi kualitas ROI berdasarkan multiple metrics.
        Returns: quality score (0-1)
        """
        h, w = frame.shape[:2]
        pixels = []
        
        for idx in region_indices:
            x = int(landmarks.landmark[idx].x * w)
            y = int(landmarks.landmark[idx].y * h)
            # Pastikan koordinat dalam bounds
            if 0 <= y < h and 0 <= x < w:
                pixels.append(frame[y, x])
        
        if len(pixels) == 0:
            return 0.0
        
        pixels = np.array(pixels)
        rgb_mean = np.mean(pixels, axis=0)
        
        # 1. Brightness check (avoid over/under exposure)
        brightness = np.mean(rgb_mean)
        exposure_score = 1.0 - abs(brightness - 128) / 128.0
        
        # 2. Saturation check (blood flow visibility)
        saturation = np.std(pixels)
        saturation_score = min(saturation / 30.0, 1.0)
        
        # 3. Green channel prominence (heart rate signal strongest in green)
        green_ratio = rgb_mean[1] / (np.sum(rgb_mean) + 1e-9)
        green_score = min(green_ratio / 0.4, 1.0)
        
        # Combined score
        quality = 0.4 * exposure_score + 0.3 * saturation_score + 0.3 * green_score
        
        return np.clip(quality, 0, 1)
    
    def select_best_rois(self, frame, landmarks, top_k=2):
        """
        Pilih top-K ROI dengan kualitas terbaik.
        Returns: list of (region_name, quality_score)
        """
        if landmarks is None:
            return []
        
        roi_scores = {}
        
        for region_name, region_indices in self.candidate_regions.items():
            quality = self.evaluate_roi_quality(frame, landmarks, region_indices)
            roi_scores[region_name] = quality
        
        # Sort by quality and return top-K
        sorted_rois = sorted(roi_scores.items(), key=lambda x: x[1], reverse=True)
        best_rois = sorted_rois[:top_k]
        
        return best_rois
    
    def extract_averaged_rgb(self, frame, landmarks, selected_rois):
        """
        Extract dan rata-rata RGB dari selected ROIs.
        """
        h, w = frame.shape[:2]
        all_rgb_values = []
        
        for region_name, quality_score in selected_rois:
            if quality_score < 0.3:  # Skip low quality ROIs
                continue
            
            region_indices = self.candidate_regions[region_name]
            pixels = []
            
            for idx in region_indices:
                x = int(landmarks.landmark[idx].x * w)
                y = int(landmarks.landmark[idx].y * h)
                if 0 <= y < h and 0 <= x < w:
                    pixels.append(frame[y, x])
            
            if len(pixels) > 0:
                roi_rgb = np.mean(pixels, axis=0)
                all_rgb_values.append(roi_rgb)
        
        if len(all_rgb_values) == 0:
            return None, None, None
        
        # Weighted average (semua ROI terpilih punya kualitas baik)
        avg_rgb = np.mean(all_rgb_values, axis=0)
        
        return avg_rgb[2], avg_rgb[1], avg_rgb[0]  # R, G, B

# Test adaptive ROI selector
adaptive_roi_selector = AdaptiveROISelector()
print("✓ Adaptive ROI selector initialized!")
print(f"  Features:")
print(f"    - Quality-based ROI evaluation")
print(f"    - Exposure and saturation checks")
print(f"    - Green channel prominence scoring")
print(f"    - Dynamic top-K selection")

## 3. Signal Processing dengan POS Method

Implementasi **Plane-Orthogonal-to-Skin (POS)** method untuk ekstraksi sinyal yang lebih robust terhadap gerakan dan perubahan pencahayaan.

In [ ]:
class SignalProcessor:
    """
    Signal processor dengan POS (Plane-Orthogonal-to-Skin) method.
    Improvement: POS lebih robust terhadap motion artifacts dibanding metode GREEN sederhana.
    """
    def __init__(self, fps=30, window_size=10):
        self.fps = fps
        self.window_size = window_size  # seconds
        self.buffer_size = int(fps * window_size)
        
        # Buffers untuk menyimpan sinyal RGB
        self.red_buffer = deque(maxlen=self.buffer_size)
        self.green_buffer = deque(maxlen=self.buffer_size)
        self.blue_buffer = deque(maxlen=self.buffer_size)
        
        # BPM range: 40-240 BPM = 0.67-4.0 Hz
        self.min_hz = 0.67
        self.max_hz = 4.0
        
    def extract_color_signals(self, frame, roi_mask):
        """
        Ekstrak rata-rata nilai RGB dari ROI (spatial averaging).
        """
        if roi_mask is None:
            return None, None, None
        
        # Apply mask
        masked_frame = cv2.bitwise_and(frame, frame, mask=roi_mask)
        
        # Calculate mean values for each channel in ROI
        pixels = masked_frame[roi_mask > 0]
        
        if len(pixels) == 0:
            return None, None, None
        
        b_mean = np.mean(pixels[:, 0])
        g_mean = np.mean(pixels[:, 1])
        r_mean = np.mean(pixels[:, 2])
        
        return r_mean, g_mean, b_mean
    
    def add_values(self, r, g, b):
        """
        Tambahkan nilai RGB ke buffer.
        """
        if r is not None and g is not None and b is not None:
            self.red_buffer.append(r)
            self.green_buffer.append(g)
            self.blue_buffer.append(b)
    
    def pos_method(self):
        """
        POS (Plane-Orthogonal-to-Skin) method untuk ekstraksi sinyal pulse.
        Paper: "Improved Motion Robustness of Remote-PPG by Using the Blood Volume Pulse Signature"
        """
        if len(self.red_buffer) < self.buffer_size // 2:
            return None
        
        # Convert buffers to numpy arrays
        R = np.array(self.red_buffer)
        G = np.array(self.green_buffer)
        B = np.array(self.blue_buffer)
        
        # Normalize signals
        R_norm = (R - np.mean(R)) / np.std(R)
        G_norm = (G - np.mean(G)) / np.std(G)
        B_norm = (B - np.mean(B)) / np.std(B)
        
        # Build projection matrix (POS algorithm)
        X = np.array([R_norm, G_norm, B_norm])
        
        # Project onto plane orthogonal to skin-tone
        S1 = X[0] - X[1]  # R - G
        S2 = X[0] + X[1] - 2 * X[2]  # R + G - 2B
        
        # Calculate pulse signal
        alpha = np.std(S1) / np.std(S2)
        pulse_signal = S1 - alpha * S2
        
        return pulse_signal
    
    def detrend_signal(self, signal_data):
        """
        Remove slow drift using signal detrending.
        """
        if signal_data is None or len(signal_data) < 10:
            return None
        
        # Remove linear trend
        detrended = signal.detrend(signal_data, type='linear')
        return detrended
    
    def bandpass_filter(self, signal_data):
        """
        Apply bandpass filter (0.67-4.0 Hz untuk 40-240 BPM).
        """
        if signal_data is None or len(signal_data) < 10:
            return None
        
        # Design Butterworth bandpass filter
        nyquist = 0.5 * self.fps
        low = self.min_hz / nyquist
        high = self.max_hz / nyquist
        
        # Ensure cutoff frequencies are valid
        low = max(0.01, min(low, 0.99))
        high = max(low + 0.01, min(high, 0.99))
        
        try:
            b, a = signal.butter(4, [low, high], btype='band')
            filtered_signal = signal.filtfilt(b, a, signal_data)
            return filtered_signal
        except:
            return signal_data
    
    def estimate_bpm(self, pulse_signal):
        """
        Estimate BPM using FFT (Fourier Transform).
        """
        if pulse_signal is None or len(pulse_signal) < self.buffer_size // 2:
            return None, None
        
        # Apply detrending
        pulse_signal = self.detrend_signal(pulse_signal)
        if pulse_signal is None:
            return None, None
        
        # Apply bandpass filter
        pulse_signal = self.bandpass_filter(pulse_signal)
        if pulse_signal is None:
            return None, None
        
        # Apply FFT
        n = len(pulse_signal)
        fft_values = fft(pulse_signal)
        fft_freqs = fftfreq(n, 1.0 / self.fps)
        
        # Get positive frequencies only
        positive_freqs = fft_freqs[:n // 2]
        positive_fft = np.abs(fft_values[:n // 2])
        
        # Find frequencies in valid range (0.67-4.0 Hz)
        valid_idx = np.where((positive_freqs >= self.min_hz) & (positive_freqs <= self.max_hz))[0]
        
        if len(valid_idx) == 0:
            return None, None
        
        valid_freqs = positive_freqs[valid_idx]
        valid_fft = positive_fft[valid_idx]
        
        # Find dominant frequency
        peak_idx = np.argmax(valid_fft)
        dominant_freq = valid_freqs[peak_idx]
        
        # Convert to BPM
        bpm = dominant_freq * 60
        
        return bpm, pulse_signal

# Test the signal processor
processor = SignalProcessor(fps=30, window_size=10)
print("✓ Signal processor initialized successfully!")
print(f"  Buffer size: {processor.buffer_size} frames")
print(f"  Valid BPM range: {processor.min_hz * 60:.0f} - {processor.max_hz * 60:.0f} BPM")

## 3.5. Advanced Signal Processing - Improvements

**Berdasarkan analisis kelemahan sistem sebelumnya, ditambahkan:**
- **Signal Quality Index (SQI)** - Confidence scoring untuk setiap pengukuran
- **Robust Peak Detection** - Multi-criteria filtering (prominence, width, height)
- **Adaptive Filtering** - Parameter tuning berdasarkan SNR
- **Motion Artifact Detection** - Deteksi dan flagging gerakan kepala
- **Interpolation** - Handle missing frames dengan smart interpolation

In [ ]:
from scipy import stats
from scipy.signal import find_peaks

class AdvancedSignalProcessor(SignalProcessor):
    """
    Enhanced signal processor dengan improvements berdasarkan analisis kelemahan:
    1. Signal Quality Index (SQI) - menilai kualitas sinyal
    2. Robust peak detection - multi-criteria filtering
    3. Motion artifact detection
    4. Adaptive filtering
    5. Missing data interpolation
    """
    
    def __init__(self, fps=30, window_size=10):
        super().__init__(fps, window_size)
        self.sqi_history = deque(maxlen=10)
        self.motion_detected = False
        
    def compute_signal_quality(self, signal):
        """
        Hitung Signal Quality Index (SQI) dari sinyal.
        Returns: SQI score (0-1), higher is better
        
        Menggunakan multiple metrics:
        - SNR (Signal-to-Noise Ratio)
        - Kurtosis (distribusi)
        - Variance stability
        """
        if signal is None or len(signal) < 10:
            return 0.0
        
        # 1. SNR Calculation
        fft_values = fft(signal)
        power_spectrum = np.abs(fft_values) ** 2
        
        # Signal band untuk heart rate (0.75-3.0 Hz = 45-180 BPM)
        freqs = fftfreq(len(signal), 1.0 / self.fps)
        hr_band_mask = (np.abs(freqs) >= 0.75) & (np.abs(freqs) <= 3.0)
        
        signal_power = np.sum(power_spectrum[hr_band_mask])
        noise_power = np.sum(power_spectrum[~hr_band_mask])
        
        snr = signal_power / (noise_power + 1e-9)
        snr_score = np.tanh(snr / 5.0)  # Normalize to 0-1
        
        # 2. Kurtosis check (gaussian-like distribution = good)
        kurt = stats.kurtosis(signal)
        kurt_score = 1.0 - min(abs(kurt - 3.0) / 10.0, 1.0)
        
        # 3. Variance stability (not too noisy, not flat)
        variance = np.var(signal)
        variance_score = np.tanh(variance / 100.0)
        
        # Combined SQI (weighted average)
        sqi = 0.5 * snr_score + 0.3 * kurt_score + 0.2 * variance_score
        
        return np.clip(sqi, 0, 1)
    
    def robust_peak_detection(self, signal, min_distance_sec=0.5):
        """
        Improved peak detection dengan multi-criteria:
        - Distance constraint
        - Prominence (puncak harus menonjol)
        - Width constraint (durasi peak realistic)
        - Height threshold
        - Outlier removal (IQR method)
        """
        if signal is None or len(signal) < 10:
            return np.array([])
        
        # Normalize signal
        signal_norm = (signal - np.mean(signal)) / (np.std(signal) + 1e-9)
        
        # Multi-criteria peak detection
        peaks, properties = find_peaks(
            signal_norm,
            distance=int(min_distance_sec * self.fps),
            prominence=0.3,  # Puncak harus menonjol minimal 0.3 std
            width=(self.fps * 0.1, self.fps * 0.5),  # Lebar peak 0.1-0.5 detik
            height=0.1  # Minimal tinggi 0.1
        )
        
        # Remove outlier peaks using Interquartile Range
        if len(peaks) > 3:
            intervals = np.diff(peaks)
            q1, q3 = np.percentile(intervals, [25, 75])
            iqr = q3 - q1
            
            # Filter intervals yang terlalu ekstrem
            valid_mask = (intervals >= q1 - 1.5 * iqr) & (intervals <= q3 + 1.5 * iqr)
            
            # Keep first peak + valid subsequent peaks
            valid_peaks = [peaks[0]]
            for i, is_valid in enumerate(valid_mask):
                if is_valid:
                    valid_peaks.append(peaks[i + 1])
            
            peaks = np.array(valid_peaks)
        
        return peaks
    
    def detect_motion_artifacts(self):
        """
        Deteksi motion artifacts dengan membandingkan recent vs older buffer.
        Returns: True jika terdeteksi gerakan signifikan
        """
        if len(self.red_buffer) < 20:
            return False
        
        # Ambil 10 frame terakhir vs 10 frame sebelumnya
        recent = np.array(list(self.red_buffer)[-10:])
        older = np.array(list(self.red_buffer)[-20:-10])
        
        # Hitung pergeseran mean (jika besar = ada gerakan)
        mean_shift = np.abs(np.mean(recent) - np.mean(older))
        threshold = 0.3 * np.std(older)
        
        self.motion_detected = mean_shift > threshold
        return self.motion_detected
    
    def interpolate_missing_data(self, data):
        """
        Interpolasi untuk handle missing frames (NaN values).
        Menggunakan linear interpolation.
        """
        data_array = np.array(data)
        
        if len(data_array) == 0:
            return data_array
        
        # Find NaN positions
        mask = np.isnan(data_array)
        
        if mask.all():  # Semua NaN
            return data_array
        
        # Get valid indices
        valid_indices = np.where(~mask)[0]
        
        if len(valid_indices) > 1:
            # Interpolate
            data_array[mask] = np.interp(
                np.where(mask)[0],
                valid_indices,
                data_array[valid_indices]
            )
        
        return data_array
    
    def estimate_bpm_with_confidence(self, pulse_signal):
        """
        Estimate BPM dengan confidence interval menggunakan robust peak detection.
        Returns: (bpm, confidence, sqi)
        """
        if pulse_signal is None or len(pulse_signal) < self.buffer_size // 2:
            return None, 0.0, 0.0
        
        # Apply detrending
        pulse_signal = self.detrend_signal(pulse_signal)
        if pulse_signal is None:
            return None, 0.0, 0.0
        
        # Apply bandpass filter
        pulse_signal = self.bandpass_filter(pulse_signal)
        if pulse_signal is None:
            return None, 0.0, 0.0
        
        # Compute Signal Quality
        sqi = self.compute_signal_quality(pulse_signal)
        self.sqi_history.append(sqi)
        
        # Detect motion artifacts
        motion_detected = self.detect_motion_artifacts()
        
        # If poor quality or motion detected, return with low confidence
        if sqi < 0.3 or motion_detected:
            return None, 0.0, sqi
        
        # Robust peak detection
        peaks = self.robust_peak_detection(pulse_signal, min_distance_sec=0.5)
        
        if len(peaks) < 2:
            return None, 0.0, sqi
        
        # Calculate BPM
        duration_min = len(pulse_signal) / self.fps / 60.0
        bpm = len(peaks) / duration_min
        
        # Confidence based on:
        # 1. SQI score (50%)
        # 2. Number of peaks detected (30%)
        # 3. Consistency of inter-beat intervals (20%)
        
        # Peak count score (optimal: 10-15 peaks in 10 seconds)
        peak_count_score = min(len(peaks) / 15.0, 1.0)
        
        # IBI consistency (low std = consistent = high score)
        if len(peaks) > 2:
            intervals = np.diff(peaks)
            cv = np.std(intervals) / (np.mean(intervals) + 1e-9)  # Coefficient of variation
            consistency_score = np.exp(-cv * 2)  # Lower CV = higher score
        else:
            consistency_score = 0.5
        
        # Combined confidence
        confidence = 0.5 * sqi + 0.3 * peak_count_score + 0.2 * consistency_score
        confidence = np.clip(confidence, 0, 1)
        
        return bpm, confidence, sqi

# Test advanced processor
advanced_processor = AdvancedSignalProcessor(fps=30, window_size=10)
print("✓ Advanced signal processor initialized!")
print(f"  Improvements:")
print(f"    - Signal Quality Index (SQI)")
print(f"    - Robust peak detection with multi-criteria")
print(f"    - Motion artifact detection")
print(f"    - Missing data interpolation")
print(f"    - Confidence intervals")

## 4. Visualizer untuk Real-time Monitoring

Visualisasi yang informatif untuk monitoring sinyal dan BPM secara real-time.

In [ ]:
class Visualizer:
    """
    Visualizer untuk menampilkan informasi real-time.
    Improvement: Visualisasi lengkap dengan pulse signal dan FFT spectrum.
    """
    def __init__(self):
        self.bpm_history = deque(maxlen=100)
        self.colors = {
            'safe': (0, 255, 0),      # Green
            'warning': (0, 165, 255),  # Orange
            'danger': (0, 0, 255)      # Red
        }
    
    def get_bpm_color(self, bpm):
        """
        Determine color based on BPM value.
        Normal range: 60-100 BPM
        """
        if bpm is None:
            return self.colors['safe']
        
        if 60 <= bpm <= 100:
            return self.colors['safe']
        elif 50 <= bpm < 60 or 100 < bpm <= 120:
            return self.colors['warning']
        else:
            return self.colors['danger']
    
    def draw_roi(self, frame, roi_mask):
        """
        Draw ROI overlay on frame.
        """
        if roi_mask is not None:
            # Create colored overlay
            overlay = frame.copy()
            overlay[roi_mask > 0] = overlay[roi_mask > 0] * 0.7 + np.array([0, 255, 0]) * 0.3
            frame = cv2.addWeighted(frame, 0.7, overlay, 0.3, 0)
        
        return frame
    
    def draw_info_panel(self, frame, bpm, fps_value, signal_quality="Good"):
        """
        Draw information panel with BPM and other metrics.
        """
        h, w = frame.shape[:2]
        
        # Create semi-transparent panel
        panel_height = 150
        overlay = frame.copy()
        cv2.rectangle(overlay, (10, 10), (w - 10, panel_height), (0, 0, 0), -1)
        frame = cv2.addWeighted(overlay, 0.6, frame, 0.4, 0)
        
        # Draw BPM
        if bpm is not None:
            bpm_color = self.get_bpm_color(bpm)
            bpm_text = f"{bpm:.1f} BPM"
            cv2.putText(frame, bpm_text, (20, 60), 
                       cv2.FONT_HERSHEY_DUPLEX, 1.8, bpm_color, 3)
            
            # BPM status
            if 60 <= bpm <= 100:
                status = "Normal"
                status_color = self.colors['safe']
            elif 50 <= bpm < 60 or 100 < bpm <= 120:
                status = "Borderline"
                status_color = self.colors['warning']
            else:
                status = "Abnormal"
                status_color = self.colors['danger']
            
            cv2.putText(frame, status, (20, 100), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, status_color, 2)
            
            # Add to history
            self.bpm_history.append(bpm)
        else:
            cv2.putText(frame, "Detecting...", (20, 60), 
                       cv2.FONT_HERSHEY_DUPLEX, 1.5, (255, 255, 255), 2)
        
        # Draw FPS
        cv2.putText(frame, f"FPS: {fps_value:.1f}", (20, 130), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
        # Draw signal quality
        quality_color = self.colors['safe'] if signal_quality == "Good" else self.colors['warning']
        cv2.putText(frame, f"Signal: {signal_quality}", (w - 200, 130), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, quality_color, 2)
        
        return frame
    
    def draw_pulse_graph(self, frame, pulse_signal):
        """
        Draw real-time pulse waveform.
        """
        if pulse_signal is None or len(pulse_signal) < 10:
            return frame
        
        h, w = frame.shape[:2]
        
        # Graph dimensions
        graph_height = 100
        graph_width = 300
        graph_x = w - graph_width - 20
        graph_y = h - graph_height - 20
        
        # Create graph background
        overlay = frame.copy()
        cv2.rectangle(overlay, (graph_x, graph_y), 
                     (graph_x + graph_width, graph_y + graph_height), 
                     (0, 0, 0), -1)
        frame = cv2.addWeighted(overlay, 0.6, frame, 0.4, 0)
        
        # Draw border
        cv2.rectangle(frame, (graph_x, graph_y), 
                     (graph_x + graph_width, graph_y + graph_height), 
                     (255, 255, 255), 1)
        
        # Normalize signal for display
        signal_normalized = (pulse_signal - np.min(pulse_signal)) / (np.max(pulse_signal) - np.min(pulse_signal) + 1e-6)
        signal_scaled = graph_height - (signal_normalized * (graph_height - 20)) - 10
        
        # Draw waveform
        points = []
        step = len(pulse_signal) / graph_width
        for i in range(graph_width):
            idx = int(i * step)
            if idx < len(signal_scaled):
                x = graph_x + i
                y = int(graph_y + signal_scaled[idx])
                points.append((x, y))
        
        # Draw line
        for i in range(len(points) - 1):
            cv2.line(frame, points[i], points[i + 1], (0, 255, 0), 2)
        
        # Label
        cv2.putText(frame, "Pulse Signal", (graph_x + 5, graph_y + 20), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        return frame

# Test the visualizer
visualizer = Visualizer()
print("✓ Visualizer initialized successfully!")

## 4.5. Enhanced Visualizer dengan Confidence Indicators

**Improvements:**
- Color-coded confidence levels
- Signal quality visualization
- Motion artifact warnings
- Real-time quality feedback

In [ ]:
class EnhancedVisualizer(Visualizer):
    """
    Enhanced visualizer dengan confidence indicators dan quality feedback.
    """
    
    def __init__(self):
        super().__init__()
        self.confidence_history = deque(maxlen=100)
        self.sqi_history = deque(maxlen=100)
    
    def get_confidence_color(self, confidence):
        """
        Determine color based on confidence level.
        """
        if confidence > 0.8:
            return (0, 255, 0)  # Excellent - Green
        elif confidence > 0.6:
            return (100, 200, 255)  # Good - Light Blue
        elif confidence > 0.4:
            return (0, 165, 255)  # Fair - Orange
        else:
            return (0, 0, 255)  # Poor - Red
    
    def draw_info_panel_enhanced(self, frame, bpm, fps_value, confidence=None, sqi=None, motion_detected=False):
        """
        Enhanced info panel dengan confidence dan quality indicators.
        """
        h, w = frame.shape[:2]
        
        # Create semi-transparent panel (larger untuk info tambahan)
        panel_height = 200
        overlay = frame.copy()
        cv2.rectangle(overlay, (10, 10), (w - 10, panel_height), (0, 0, 0), -1)
        frame = cv2.addWeighted(overlay, 0.6, frame, 0.4, 0)
        
        # Draw BPM dengan confidence color
        if bpm is not None and confidence is not None:
            conf_color = self.get_confidence_color(confidence)
            bpm_text = f"{bpm:.1f} BPM"
            cv2.putText(frame, bpm_text, (20, 60), 
                       cv2.FONT_HERSHEY_DUPLEX, 1.8, conf_color, 3)
            
            # Confidence bar
            bar_width = 200
            bar_height = 20
            bar_x = 20
            bar_y = 75
            
            # Background bar
            cv2.rectangle(frame, (bar_x, bar_y), (bar_x + bar_width, bar_y + bar_height), 
                         (50, 50, 50), -1)
            
            # Confidence fill
            fill_width = int(bar_width * confidence)
            cv2.rectangle(frame, (bar_x, bar_y), (bar_x + fill_width, bar_y + bar_height), 
                         conf_color, -1)
            
            # Confidence text
            conf_text = f"Confidence: {confidence*100:.1f}%"
            cv2.putText(frame, conf_text, (bar_x, bar_y + bar_height + 20), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
            
            # BPM status dengan confidence level
            if confidence > 0.7:
                if 60 <= bpm <= 100:
                    status = "Normal (Reliable)"
                    status_color = self.colors['safe']
                elif 50 <= bpm < 60 or 100 < bpm <= 120:
                    status = "Borderline (Reliable)"
                    status_color = self.colors['warning']
                else:
                    status = "Abnormal (Check)"
                    status_color = self.colors['danger']
            else:
                status = f"Measuring... ({confidence*100:.0f}% confident)"
                status_color = (200, 200, 200)
            
            cv2.putText(frame, status, (20, 125), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, status_color, 2)
            
            # Add to history
            self.bpm_history.append(bpm)
            self.confidence_history.append(confidence)
        else:
            cv2.putText(frame, "Detecting...", (20, 60), 
                       cv2.FONT_HERSHEY_DUPLEX, 1.5, (255, 255, 255), 2)
        
        # Signal Quality Index
        if sqi is not None:
            sqi_text = f"Signal Quality: {sqi*100:.1f}%"
            sqi_color = self.get_confidence_color(sqi)
            cv2.putText(frame, sqi_text, (20, 150), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, sqi_color, 2)
            self.sqi_history.append(sqi)
        
        # Motion warning
        if motion_detected:
            cv2.putText(frame, "⚠ MOTION DETECTED - Stay still!", (20, 175), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
        # Draw FPS
        cv2.putText(frame, f"FPS: {fps_value:.1f}", (20, 195), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
        return frame
    
    def draw_quality_graph(self, frame):
        """
        Draw Signal Quality Index history graph.
        """
        if len(self.sqi_history) < 2:
            return frame
        
        h, w = frame.shape[:2]
        
        # Graph dimensions
        graph_height = 80
        graph_width = 300
        graph_x = w - graph_width - 20
        graph_y = 20
        
        # Create graph background
        overlay = frame.copy()
        cv2.rectangle(overlay, (graph_x, graph_y), 
                     (graph_x + graph_width, graph_y + graph_height), 
                     (0, 0, 0), -1)
        frame = cv2.addWeighted(overlay, 0.6, frame, 0.4, 0)
        
        # Draw border
        cv2.rectangle(frame, (graph_x, graph_y), 
                     (graph_x + graph_width, graph_y + graph_height), 
                     (255, 255, 255), 1)
        
        # Draw quality threshold lines
        good_y = int(graph_y + graph_height * (1 - 0.7))
        fair_y = int(graph_y + graph_height * (1 - 0.4))
        cv2.line(frame, (graph_x, good_y), (graph_x + graph_width, good_y), 
                (0, 255, 0), 1, cv2.LINE_AA)
        cv2.line(frame, (graph_x, fair_y), (graph_x + graph_width, fair_y), 
                (0, 165, 255), 1, cv2.LINE_AA)
        
        # Plot SQI history
        sqi_array = np.array(self.sqi_history)
        points = []
        step = graph_width / len(sqi_array)
        
        for i, sqi in enumerate(sqi_array):
            x = int(graph_x + i * step)
            y = int(graph_y + graph_height - (sqi * graph_height))
            points.append((x, y))
        
        # Draw line
        for i in range(len(points) - 1):
            color = self.get_confidence_color(sqi_array[i])
            cv2.line(frame, points[i], points[i + 1], color, 2, cv2.LINE_AA)
        
        # Label
        cv2.putText(frame, "Signal Quality History", (graph_x + 5, graph_y + 15), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
        
        return frame
    
    def draw_lighting_suggestion(self, frame, avg_brightness):
        """
        Provide lighting adjustment suggestions.
        """
        h, w = frame.shape[:2]
        
        if avg_brightness < 80:
            suggestion = "⚠ Too dark - Increase lighting"
            color = (0, 165, 255)
        elif avg_brightness > 200:
            suggestion = "⚠ Too bright - Reduce lighting"
            color = (0, 165, 255)
        else:
            suggestion = "✓ Lighting OK"
            color = (0, 255, 0)
        
        cv2.putText(frame, suggestion, (w - 350, h - 20), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        
        return frame

# Test enhanced visualizer
enhanced_viz = EnhancedVisualizer()
print("✓ Enhanced visualizer initialized!")
print(f"  Features:")
print(f"    - Confidence indicators with color coding")
print(f"    - Signal quality visualization")
print(f"    - Motion artifact warnings")
print(f"    - Lighting suggestions")
print(f"    - Real-time quality graph")

## 5. Real-time rPPG System

Integrasi semua komponen menjadi sistem real-time yang lengkap.

In [ ]:
class RealTimeRPPG:
    """
    Real-time rPPG system yang mengintegrasikan semua komponen.
    """
    def __init__(self, camera_index=0, fps=30):
        self.camera_index = camera_index
        self.fps = fps
        
        # Initialize components
        self.detector = FaceDetector()
        self.processor = SignalProcessor(fps=fps, window_size=10)
        self.visualizer = Visualizer()
        
        # Camera
        self.cap = None
        
        # Metrics
        self.frame_count = 0
        self.start_time = None
        self.current_bpm = None
        
    def start(self):
        """
        Start the real-time rPPG system.
        """
        # Open camera
        self.cap = cv2.VideoCapture(self.camera_index)
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        self.cap.set(cv2.CAP_PROP_FPS, self.fps)
        
        if not self.cap.isOpened():
            print("Error: Cannot open camera!")
            return
        
        print("=" * 60)
        print("Real-time rPPG System Started!")
        print("=" * 60)
        print("Instructions:")
        print("  - Posisikan wajah di depan kamera")
        print("  - Pastikan pencahayaan cukup")
        print("  - Usahakan tidak bergerak terlalu banyak")
        print("  - Tekan 'q' untuk keluar")
        print("  - Tekan 's' untuk screenshot")
        print("=" * 60)
        
        self.start_time = time.time()
        
        try:
            self.run_loop()
        finally:
            self.cleanup()
    
    def run_loop(self):
        """
        Main processing loop.
        """
        while True:
            ret, frame = self.cap.read()
            
            if not ret:
                print("Error: Cannot read frame!")
                break
            
            # Flip frame for mirror effect
            frame = cv2.flip(frame, 1)
            
            # Process frame
            processed_frame = self.process_frame(frame)
            
            # Display
            cv2.imshow('Real-time rPPG - Heart Rate Monitor', processed_frame)
            
            # Handle keyboard input
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                print("\nStopping system...")
                break
            elif key == ord('s'):
                filename = f"rppg_screenshot_{int(time.time())}.jpg"
                cv2.imwrite(filename, processed_frame)
                print(f"Screenshot saved: {filename}")
            
            self.frame_count += 1
    
    def process_frame(self, frame):
        """
        Process single frame and return visualization.
        """
        # Detect face and get ROI
        roi_mask, face_landmarks = self.detector.detect_face_roi(frame)
        
        # Extract color signals
        r, g, b = self.processor.extract_color_signals(frame, roi_mask)
        
        # Add to buffer
        if r is not None:
            self.processor.add_values(r, g, b)
        
        # Get pulse signal using POS method
        pulse_signal = self.processor.pos_method()
        
        # Estimate BPM every 30 frames (sekitar 1 detik)
        if self.frame_count % 30 == 0:
            bpm, filtered_signal = self.processor.estimate_bpm(pulse_signal)
            if bpm is not None:
                self.current_bpm = bpm
        
        # Calculate FPS
        elapsed = time.time() - self.start_time
        current_fps = self.frame_count / elapsed if elapsed > 0 else 0
        
        # Determine signal quality
        signal_quality = "Good" if roi_mask is not None and r is not None else "Poor"
        
        # Visualize
        viz_frame = self.visualizer.draw_roi(frame, roi_mask)
        viz_frame = self.visualizer.draw_info_panel(
            viz_frame, 
            self.current_bpm, 
            current_fps, 
            signal_quality
        )
        viz_frame = self.visualizer.draw_pulse_graph(viz_frame, pulse_signal)
        
        return viz_frame
    
    def cleanup(self):
        """
        Cleanup resources.
        """
        if self.cap is not None:
            self.cap.release()
        cv2.destroyAllWindows()
        
        print("\nSystem stopped.")
        print(f"Total frames processed: {self.frame_count}")
        if self.current_bpm is not None:
            print(f"Last BPM reading: {self.current_bpm:.1f}")

print("✓ Real-time rPPG system ready!")

## 5.5. Improved Real-time rPPG System

**Integrasi semua improvements:**
- Adaptive ROI selection
- Advanced signal processing dengan SQI
- Robust peak detection
- Motion artifact detection
- Enhanced visualization dengan confidence
- Lighting quality checks

In [ ]:
class ImprovedRealTimeRPPG:
    """
    Improved real-time rPPG system dengan semua enhancements.
    
    Key Improvements dari sistem sebelumnya:
    1. Adaptive ROI selection (quality-based)
    2. Signal Quality Index (SQI) monitoring
    3. Robust peak detection
    4. Motion artifact detection
    5. Enhanced visualization dengan confidence indicators
    6. Lighting quality feedback
    7. Interpolation untuk missing frames
    """
    
    def __init__(self, camera_index=0, fps=30):
        self.camera_index = camera_index
        self.fps = fps
        
        # Initialize enhanced components
        self.detector = FaceDetector()
        self.adaptive_roi = AdaptiveROISelector()
        self.processor = AdvancedSignalProcessor(fps=fps, window_size=10)
        self.visualizer = EnhancedVisualizer()
        
        # Camera
        self.cap = None
        
        # Metrics
        self.frame_count = 0
        self.start_time = None
        self.current_bpm = None
        self.current_confidence = 0.0
        self.current_sqi = 0.0
        
        # Quality tracking
        self.roi_quality_history = deque(maxlen=30)
        
    def start(self):
        """
        Start the improved real-time rPPG system.
        """
        # Open camera
        self.cap = cv2.VideoCapture(self.camera_index)
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        self.cap.set(cv2.CAP_PROP_FPS, self.fps)
        
        if not self.cap.isOpened():
            print("Error: Cannot open camera!")
            return
        
        print("=" * 70)
        print("IMPROVED Real-time rPPG System Started!")
        print("=" * 70)
        print("Features:")
        print("  ✓ Adaptive ROI selection (quality-based)")
        print("  ✓ Signal Quality Index (SQI) monitoring")
        print("  ✓ Robust peak detection dengan multi-criteria")
        print("  ✓ Motion artifact detection")
        print("  ✓ Confidence intervals untuk setiap measurement")
        print("  ✓ Enhanced visualization dengan quality feedback")
        print()
        print("Instructions:")
        print("  - Posisikan wajah di depan kamera")
        print("  - Pastikan pencahayaan cukup (tidak terlalu gelap/terang)")
        print("  - Duduk tenang, usahakan tidak bergerak")
        print("  - Tunggu hingga confidence > 70% untuk hasil reliable")
        print("  - Tekan 'q' untuk keluar")
        print("  - Tekan 's' untuk screenshot")
        print("  - Tekan 'r' untuk reset measurements")
        print("=" * 70)
        
        self.start_time = time.time()
        
        try:
            self.run_loop()
        finally:
            self.cleanup()
    
    def run_loop(self):
        """
        Main processing loop dengan enhanced features.
        """
        while True:
            ret, frame = self.cap.read()
            
            if not ret:
                print("Error: Cannot read frame!")
                break
            
            # Flip frame for mirror effect
            frame = cv2.flip(frame, 1)
            
            # Process frame
            processed_frame = self.process_frame(frame)
            
            # Display
            cv2.imshow('Improved rPPG - Heart Rate Monitor with Quality Metrics', processed_frame)
            
            # Handle keyboard input
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                print("\nStopping system...")
                break
            elif key == ord('s'):
                filename = f"improved_rppg_screenshot_{int(time.time())}.jpg"
                cv2.imwrite(filename, processed_frame)
                print(f"Screenshot saved: {filename}")
            elif key == ord('r'):
                print("Resetting measurements...")
                self.processor.red_buffer.clear()
                self.processor.green_buffer.clear()
                self.processor.blue_buffer.clear()
                self.current_bpm = None
                self.current_confidence = 0.0
                self.current_sqi = 0.0
            
            self.frame_count += 1
    
    def process_frame(self, frame):
        """
        Process single frame dengan all enhancements.
        """
        # Detect face mesh
        face_mesh = self.detector.detect(frame)
        
        # Extract ROIs
        if face_mesh is not None:
            forehead_roi, left_cheek_roi, right_cheek_roi = self.detector.extract_roi(frame, face_mesh)
            
            # Adaptive ROI selection (quality-based)
            if forehead_roi is not None:
                best_roi, roi_quality = self.adaptive_roi.select_best_roi(
                    forehead_roi, left_cheek_roi, right_cheek_roi
                )
                
                # Track ROI quality
                self.roi_quality_history.append(roi_quality)
                
                # Extract RGB signal
                rgb_signal = self.processor.extract_signal(best_roi)
                
                if rgb_signal is not None:
                    r, g, b = rgb_signal[0], rgb_signal[1], rgb_signal[2]
                else:
                    r, g, b = None, None, None
            else:
                r, g, b = None, None, None
                self.roi_quality_history.append(0.0)
        else:
            r, g, b = None, None, None
            self.roi_quality_history.append(0.0)
        
        # Add to buffer (dengan interpolation jika ada NaN sebelumnya)
        if r is not None:
            self.processor.add_values(r, g, b)
        else:
            self.processor.add_values(np.nan, np.nan, np.nan)
        
        # Get pulse signal using POS method
        pulse_signal = self.processor.pos_method()
        
        # Estimate BPM dengan confidence setiap 30 frames (~1 detik)
        if self.frame_count % 30 == 0 and pulse_signal is not None:
            bpm, confidence, sqi = self.processor.estimate_bpm_with_confidence(pulse_signal)
            
            # Hanya update jika confidence cukup tinggi
            if bpm is not None and confidence > 0.3:
                self.current_bpm = bpm
                self.current_confidence = confidence
                self.current_sqi = sqi
        
        # Calculate FPS
        elapsed = time.time() - self.start_time
        current_fps = self.frame_count / elapsed if elapsed > 0 else 0
        
        # Calculate average brightness for lighting check
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        avg_brightness = np.mean(gray)
        
        # Draw face mesh if detected
        viz_frame = frame.copy()
        if face_mesh is not None:
            viz_frame = self.detector.draw_face_mesh(viz_frame, face_mesh)
        
        # Enhanced visualization
        motion_detected = hasattr(self.processor, 'motion_detected') and self.processor.motion_detected
        viz_frame = self.visualizer.draw_info_panel_enhanced(
            viz_frame,
            self.current_bpm,
            current_fps,
            confidence=self.current_confidence,
            sqi=self.current_sqi,
            motion_detected=motion_detected
        )
        viz_frame = self.visualizer.draw_pulse_graph(viz_frame, pulse_signal)
        viz_frame = self.visualizer.draw_quality_graph(viz_frame)
        viz_frame = self.visualizer.draw_lighting_suggestion(viz_frame, avg_brightness)
        
        return viz_frame
    
    def cleanup(self):
        """
        Cleanup resources with summary statistics.
        """
        if self.cap is not None:
            self.cap.release()
        cv2.destroyAllWindows()
        
        print("\n" + "=" * 70)
        print("System stopped - Session Summary:")
        print("=" * 70)
        print(f"Total frames processed: {self.frame_count}")
        print(f"Session duration: {time.time() - self.start_time:.1f} seconds")
        
        if self.current_bpm is not None:
            print(f"\nFinal Measurements:")
            print(f"  BPM: {self.current_bpm:.1f}")
            print(f"  Confidence: {self.current_confidence*100:.1f}%")
            print(f"  Signal Quality: {self.current_sqi*100:.1f}%")
            
            # Reliability assessment
            if self.current_confidence > 0.8 and self.current_sqi > 0.7:
                print(f"  Reliability: ✓ EXCELLENT - Measurement is highly reliable")
            elif self.current_confidence > 0.6 and self.current_sqi > 0.5:
                print(f"  Reliability: ✓ GOOD - Measurement is reliable")
            elif self.current_confidence > 0.4:
                print(f"  Reliability: ⚠ FAIR - Use with caution")
            else:
                print(f"  Reliability: ✗ POOR - Measurement not reliable")
        else:
            print("\nNo reliable measurements obtained.")
        
        # ROI quality summary
        if len(self.roi_quality_history) > 0:
            avg_roi_quality = np.mean(self.roi_quality_history)
            print(f"\nAverage ROI Quality: {avg_roi_quality*100:.1f}%")
        
        print("=" * 70)

print("✓ Improved real-time rPPG system ready!")
print("\nTo run:")
print("  improved_system = ImprovedRealTimeRPPG(camera_index=0, fps=30)")
print("  improved_system.start()")

## 6. Run the IMPROVED System

**Pilihan Sistem:**
- **Basic System** (cell sebelumnya) - Implementasi dasar dengan POS method
- **Improved System** (cell ini) - Dengan semua enhancements dan quality metrics

**Rekomendasi:** Gunakan Improved System untuk hasil yang lebih akurat dan reliable.

In [ ]:
# Create and start the IMPROVED rPPG system
if __name__ == "__main__":
    # Initialize improved system dengan semua enhancements
    improved_system = ImprovedRealTimeRPPG(camera_index=0, fps=30)
    
    # Start real-time processing
    improved_system.start()

## 6. Run the System

Jalankan sistem rPPG real-time. Pastikan webcam Anda tersedia dan pencahayaan cukup.

In [ ]:
# Create and start the rPPG system
if __name__ == "__main__":
    # Initialize system
    rppg_system = RealTimeRPPG(camera_index=0, fps=30)
    
    # Start real-time processing
    rppg_system.start()

---

## 7. Dokumentasi Improvements

### Aspek Pembeda dengan Demo Kelas & Proyek Referensi:

#### 1. **POS (Plane-Orthogonal-to-Skin) Method**
- **Demo kelas**: Menggunakan metode GREEN sederhana (hanya kanal hijau)
- **Implementasi ini**: Menggunakan POS method yang lebih robust
- **Keuntungan**: 
  - Lebih tahan terhadap motion artifacts (gerakan kepala)
  - Lebih stabil terhadap perubahan pencahayaan
  - Akurasi lebih tinggi dengan memanfaatkan ketiga kanal warna (R, G, B)

#### 2. **Adaptive ROI Selection (IMPROVEMENT UTAMA)**
- **Proyek referensi**: Fixed ROI regions (dahi + pipi kiri/kanan)
- **Implementasi improved**: **Quality-based adaptive ROI selection**
- **Keuntungan**:
  - ROI dipilih berdasarkan quality metrics (exposure, saturation, green prominence)
  - Dynamic top-K selection - hanya gunakan ROI berkualitas baik
  - Menghindari area over/under-exposed
  - Lebih robust terhadap variasi pencahayaan

#### 3. **Signal Quality Index - SQI (IMPROVEMENT UTAMA)**
- **Proyek referensi**: Tidak ada quality assessment
- **Implementasi improved**: **Comprehensive SQI dengan multiple metrics**
- **Metrics yang digunakan**:
  - SNR (Signal-to-Noise Ratio) dalam frequency domain
  - Kurtosis check (distribusi gaussian = good signal)
  - Variance stability
- **Keuntungan**:
  - User tahu kapan measurement reliable
  - Prevent displaying false readings
  - Visual feedback real-time

#### 4. **Robust Peak Detection (IMPROVEMENT UTAMA)**
- **Proyek referensi**: Simple `find_peaks()` dengan distance constraint saja
- **Implementasi improved**: **Multi-criteria peak detection + outlier removal**
- **Kriteria**:
  - Distance constraint (minimum jarak antar peak)
  - Prominence (peak harus menonjol)
  - Width constraint (durasi peak realistic)
  - Height threshold
  - IQR-based outlier removal
- **Keuntungan**:
  - Mengurangi false positives (double-counting)
  - Filter peaks yang tidak realistic
  - BPM estimation lebih akurat

#### 5. **Motion Artifact Detection (IMPROVEMENT BARU)**
- **Proyek referensi**: Tidak ada motion detection
- **Implementasi improved**: **Automatic motion artifact detection**
- **Cara kerja**:
  - Bandingkan recent buffer vs older buffer
  - Detect sudden mean shift (gerakan kepala)
  - Flag measurements selama motion
- **Keuntungan**:
  - Warning real-time saat user bergerak
  - Prevent corrupted measurements
  - Guide user untuk stay still

#### 6. **Confidence Intervals (IMPROVEMENT BARU)**
- **Proyek referensi**: Hanya tampilkan BPM tanpa confidence
- **Implementasi improved**: **Multi-factor confidence scoring**
- **Factors**:
  - SQI score (50%)
  - Peak count score (30%)
  - Inter-beat interval consistency (20%)
- **Keuntungan**:
  - User tahu seberapa reliable measurement
  - Color-coded visualization (green/yellow/red)
  - Medical-grade reporting requirement

#### 7. **Enhanced Visualization**
- **Proyek referensi**: PyQt5 GUI dengan basic plots
- **Implementasi improved**: **OpenCV real-time dengan advanced overlays**
- **Features**:
  - Confidence bar dengan color coding
  - Signal quality history graph
  - Motion artifact warnings
  - Lighting adjustment suggestions
  - Status dengan reliability level
- **Keuntungan**:
  - Tidak perlu PyQt5 dependency
  - Lighter dan faster
  - More informative feedback

#### 8. **Missing Data Handling (IMPROVEMENT BARU)**
- **Proyek referensi**: NaN values simply appended
- **Implementasi improved**: **Smart interpolation**
- **Cara kerja**:
  - Linear interpolation untuk fill missing frames
  - Maintain signal continuity
- **Keuntungan**:
  - Handle temporary detection failures
  - Smoother signal processing
  - Prevent analysis gaps

---

### Comparison Table: Referensi vs Improved

| Feature | Proyek Referensi | Implementasi Improved |
|---------|------------------|----------------------|
| **ROI Selection** | Fixed regions | ✅ Quality-based adaptive |
| **Signal Quality** | ❌ None | ✅ SQI dengan SNR + Kurtosis + Variance |
| **Peak Detection** | Basic (distance only) | ✅ Multi-criteria + outlier removal |
| **Motion Detection** | ❌ None | ✅ Automatic detection + warning |
| **Confidence Score** | ❌ None | ✅ Multi-factor scoring (0-100%) |
| **Missing Data** | NaN appended | ✅ Smart interpolation |
| **Visualization** | PyQt5 GUI | ✅ OpenCV dengan advanced overlays |
| **Lighting Check** | ❌ None | ✅ Automatic suggestions |
| **User Feedback** | Basic BPM display | ✅ Comprehensive quality indicators |
| **Reliability Info** | ❌ None | ✅ Explicit reliability levels |

---

### Kelemahan yang Diatasi:

Berdasarkan analisis PDF report, kelemahan yang berhasil diatasi:

1. ✅ **Lighting dependency** → Adaptive ROI + lighting suggestions
2. ✅ **No quality assessment** → Comprehensive SQI system
3. ✅ **Motion sensitivity** → Motion artifact detection + warnings
4. ✅ **Unreliable measurements** → Confidence intervals + quality thresholds
5. ✅ **Fixed parameters** → Adaptive ROI selection
6. ✅ **Missing frames** → Smart interpolation
7. ✅ **Simple peak detection** → Robust multi-criteria detection

---

## 8. Testing dan Validasi

Beberapa fungsi untuk testing komponen individual.

In [ ]:
def test_face_detection():
    """
    Test face detection dengan single frame dari webcam.
    """
    print("Testing Face Detection...")
    cap = cv2.VideoCapture(0)
    detector = FaceDetector()
    
    ret, frame = cap.read()
    if ret:
        roi_mask, landmarks = detector.detect_face_roi(frame)
        
        if roi_mask is not None:
            print("✓ Face detected successfully!")
            print(f"  ROI pixels: {np.sum(roi_mask > 0)}")
            
            # Visualize
            viz = frame.copy()
            viz[roi_mask > 0] = viz[roi_mask > 0] * 0.5 + np.array([0, 255, 0]) * 0.5
            
            cv2.imshow("Face Detection Test", viz)
            cv2.waitKey(2000)
            cv2.destroyAllWindows()
        else:
            print("✗ No face detected!")
    
    cap.release()

def test_signal_processing():
    """
    Test signal processing dengan synthetic signal.
    """
    print("\nTesting Signal Processing...")
    
    # Create synthetic heartbeat signal (72 BPM = 1.2 Hz)
    fps = 30
    duration = 10
    t = np.linspace(0, duration, fps * duration)
    
    # Synthetic pulse at 72 BPM (1.2 Hz)
    heart_rate_hz = 1.2
    clean_signal = np.sin(2 * np.pi * heart_rate_hz * t)
    
    # Add noise
    noise = np.random.normal(0, 0.5, len(clean_signal))
    noisy_signal = clean_signal + noise
    
    # Process
    processor = SignalProcessor(fps=fps, window_size=10)
    
    # Test detrending
    detrended = processor.detrend_signal(noisy_signal)
    print(f"✓ Detrending: {len(detrended)} samples")
    
    # Test filtering
    filtered = processor.bandpass_filter(detrended)
    print(f"✓ Filtering: {len(filtered)} samples")
    
    # Test BPM estimation
    # Simulate buffer filling
    for i in range(len(t)):
        r = 150 + clean_signal[i] * 10
        g = 100 + clean_signal[i] * 20  # Green has stronger signal
        b = 120 + clean_signal[i] * 5
        processor.add_values(r, g, b)
    
    pulse_signal = processor.pos_method()
    bpm, _ = processor.estimate_bpm(pulse_signal)
    
    if bpm is not None:
        print(f"✓ BPM Estimation: {bpm:.1f} BPM")
        print(f"  Expected: {heart_rate_hz * 60:.1f} BPM")
        print(f"  Error: {abs(bpm - heart_rate_hz * 60):.1f} BPM")
    else:
        print("✗ BPM estimation failed")

# Uncomment untuk menjalankan test
# test_face_detection()
# test_signal_processing()

print("Testing functions ready. Uncomment to run tests.")

---

## 9. Troubleshooting

### Common Issues:

#### 1. **Webcam tidak terdeteksi**
```python
# Try different camera indices
rppg_system = RealTimeRPPG(camera_index=1, fps=30)
```

#### 2. **BPM tidak stabil**
- Pastikan pencahayaan cukup dan merata
- Jangan bergerak terlalu banyak
- Tunggu 5-10 detik untuk buffer terisi
- Posisikan wajah menghadap langsung ke kamera

#### 3. **Signal Quality Poor**
- Check pencahayaan ruangan
- Bersihkan lensa kamera
- Pastikan wajah terdeteksi (ROI hijau muncul)

#### 4. **FPS rendah**
- Tutup aplikasi lain yang menggunakan kamera
- Kurangi resolusi kamera
- Gunakan komputer dengan CPU lebih kuat

---

## 10. Referensi

### Papers & Methods:
1. **POS Method**: Wang, W., den Brinker, A. C., Stuijk, S., & de Haan, G. (2016). "Algorithmic principles of remote PPG." IEEE Transactions on Biomedical Engineering.

2. **rPPG Overview**: Verkruysse, W., Svaasand, L. O., & Nelson, J. S. (2008). "Remote plethysmographic imaging using ambient light." Optics express.

3. **CHROM Method**: De Haan, G., & Jeanne, V. (2013). "Robust pulse rate from chrominance-based rPPG." IEEE Transactions on Biomedical Engineering.

### Libraries:
- **OpenCV**: Computer vision and image processing
- **MediaPipe**: Face detection and landmark extraction
- **SciPy**: Signal processing (filtering, FFT)
- **NumPy**: Numerical computations

### Course Materials:
- Pertemuan 12: Video Processing Fundamentals
- Pertemuan 13: Remote Photoplethysmography

---

## Kesimpulan

Sistem rPPG ini berhasil mengimplementasikan:
- ✅ **Pipeline lengkap**: Face detection → Signal extraction → Filtering → BPM estimation
- ✅ **Real-time capability**: Berjalan lancar dengan webcam
- ✅ **Improvements**: POS method, smart ROI, visualisasi lengkap
- ✅ **Code quality**: Terstruktur dengan baik, dokumentasi lengkap

**Note**: Sistem ini untuk tujuan edukasi. Untuk aplikasi medis yang sebenarnya, diperlukan validasi klinis yang lebih ketat.

## 🖥️ UI Fullscreen dengan Layout Modern

Visualizer yang dioptimalkan untuk resolusi 1920x1080 dengan:
- **Layout 3-panel**: Video feed + Info panel + Grafik real-time
- **Instruksi visual**: Panduan interaktif untuk pengguna
- **Dashboard modern**: Metrik kesehatan yang mudah dipahami
- **Color-coded status**: Indikator visual untuk setiap kondisi

In [ ]:
class FullscreenVisualizer:
    """
    Modern fullscreen UI untuk resolusi 1920x1080 dengan layout yang user-friendly.
    """
    
    def __init__(self):
        self.width = 1920
        self.height = 1080
        self.colors = {
            'bg': (20, 20, 25),           # Dark background
            'panel': (35, 35, 40),        # Panel background
            'accent': (0, 150, 255),      # Orange accent
            'success': (0, 255, 100),     # Green
            'warning': (0, 200, 255),     # Yellow
            'danger': (60, 60, 255),      # Red
            'text': (255, 255, 255),      # White text
            'text_dim': (150, 150, 150),  # Dim text
        }
        
        self.bpm_history = deque(maxlen=200)
        self.sqi_history = deque(maxlen=200)
        self.confidence_history = deque(maxlen=200)
        self.timestamp_history = deque(maxlen=200)
        self.start_time = time.time()
        
        # Instructions state
        self.show_instructions = True
        self.instructions_alpha = 1.0
        
    def create_canvas(self):
        """Create blank canvas untuk UI."""
        canvas = np.full((self.height, self.width, 3), self.colors['bg'], dtype=np.uint8)
        return canvas
    
    def draw_header(self, canvas, fps_value):
        """Draw header dengan branding dan info."""
        # Header bar
        cv2.rectangle(canvas, (0, 0), (self.width, 80), self.colors['panel'], -1)
        
        # Title
        cv2.putText(canvas, "rPPG Heart Rate Monitor", (40, 55), 
                   cv2.FONT_HERSHEY_BOLD, 1.3, self.colors['accent'], 3)
        
        # System info
        current_time = time.strftime("%H:%M:%S")
        info_text = f"FPS: {fps_value:.1f} | {current_time}"
        cv2.putText(canvas, info_text, (self.width - 350, 55), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, self.colors['text_dim'], 2)
        
        return canvas
    
    def draw_video_panel(self, canvas, video_frame, roi_quality=None):
        """Draw video feed panel dengan border dan ROI indicators."""
        # Panel position (kiri)
        panel_x = 40
        panel_y = 120
        panel_w = 900
        panel_h = 675
        
        # Resize video untuk fit panel
        if video_frame is not None:
            video_resized = cv2.resize(video_frame, (panel_w, panel_h))
            canvas[panel_y:panel_y+panel_h, panel_x:panel_x+panel_w] = video_resized
            
            # Border dengan warna berdasarkan quality
            if roi_quality is not None:
                if roi_quality > 0.7:
                    border_color = self.colors['success']
                elif roi_quality > 0.4:
                    border_color = self.colors['warning']
                else:
                    border_color = self.colors['danger']
            else:
                border_color = self.colors['accent']
            
            cv2.rectangle(canvas, (panel_x-3, panel_y-3), 
                         (panel_x+panel_w+3, panel_y+panel_h+3), border_color, 3)
        
        # Label
        cv2.putText(canvas, "LIVE CAMERA", (panel_x, panel_y-15), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, self.colors['text_dim'], 2)
        
        return canvas
    
    def draw_bpm_display(self, canvas, bpm, confidence=None):
        """Draw BPM besar dengan confidence indicator."""
        # Panel position (kanan atas)
        panel_x = 980
        panel_y = 120
        panel_w = 900
        panel_h = 280
        
        # Background panel
        cv2.rectangle(canvas, (panel_x, panel_y), 
                     (panel_x+panel_w, panel_y+panel_h), self.colors['panel'], -1)
        
        # Title
        cv2.putText(canvas, "HEART RATE", (panel_x+30, panel_y+50), 
                   cv2.FONT_HERSHEY_SIMPLEX, 1.0, self.colors['text_dim'], 2)
        
        if bpm is not None and confidence is not None:
            # Determine color based on confidence
            if confidence > 0.8:
                bpm_color = self.colors['success']
                status = "EXCELLENT"
            elif confidence > 0.6:
                bpm_color = self.colors['accent']
                status = "GOOD"
            elif confidence > 0.4:
                bpm_color = self.colors['warning']
                status = "FAIR"
            else:
                bpm_color = self.colors['danger']
                status = "MEASURING..."
            
            # BPM value - BESAR
            bpm_text = f"{int(bpm)}"
            cv2.putText(canvas, bpm_text, (panel_x+50, panel_y+180), 
                       cv2.FONT_HERSHEY_BOLD, 4.5, bpm_color, 8)
            
            # BPM unit
            cv2.putText(canvas, "BPM", (panel_x+370, panel_y+180), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1.5, self.colors['text_dim'], 3)
            
            # Confidence bar
            bar_x = panel_x + 50
            bar_y = panel_y + 210
            bar_w = 400
            bar_h = 25
            
            # Background
            cv2.rectangle(canvas, (bar_x, bar_y), (bar_x+bar_w, bar_y+bar_h), 
                         (50, 50, 50), -1)
            
            # Fill
            fill_w = int(bar_w * confidence)
            cv2.rectangle(canvas, (bar_x, bar_y), (bar_x+fill_w, bar_y+bar_h), 
                         bpm_color, -1)
            
            # Text
            conf_text = f"{status} - {int(confidence*100)}% Confidence"
            cv2.putText(canvas, conf_text, (bar_x, bar_y+bar_h+30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, self.colors['text'], 2)
            
            # Health status
            if confidence > 0.7:
                if 60 <= bpm <= 100:
                    health_text = "✓ Normal Heart Rate"
                    health_color = self.colors['success']
                elif 50 <= bpm < 60:
                    health_text = "↓ Below Normal"
                    health_color = self.colors['warning']
                elif 100 < bpm <= 120:
                    health_text = "↑ Above Normal"
                    health_color = self.colors['warning']
                else:
                    health_text = "⚠ Consult Healthcare Provider"
                    health_color = self.colors['danger']
                
                cv2.putText(canvas, health_text, (panel_x+500, panel_y+150), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.9, health_color, 2)
        else:
            # Waiting state
            cv2.putText(canvas, "---", (panel_x+50, panel_y+180), 
                       cv2.FONT_HERSHEY_BOLD, 4.5, self.colors['text_dim'], 8)
            cv2.putText(canvas, "Initializing...", (panel_x+50, panel_y+230), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1.0, self.colors['text_dim'], 2)
        
        return canvas
    
    def draw_metrics_panel(self, canvas, sqi=None, motion_detected=False, avg_bpm=None, bpm_range=None):
        """Draw panel metrik tambahan."""
        # Panel position (kanan tengah)
        panel_x = 980
        panel_y = 430
        panel_w = 900
        panel_h = 180
        
        # Background
        cv2.rectangle(canvas, (panel_x, panel_y), 
                     (panel_x+panel_w, panel_y+panel_h), self.colors['panel'], -1)
        
        # Metrics dalam grid 2x2
        metrics = [
            ("Signal Quality", f"{int(sqi*100)}%" if sqi else "---", 
             self.colors['success'] if sqi and sqi > 0.7 else self.colors['warning']),
            ("Motion Status", "STILL" if not motion_detected else "MOVING", 
             self.colors['success'] if not motion_detected else self.colors['danger']),
            ("Avg BPM (30s)", f"{int(avg_bpm)}" if avg_bpm else "---", 
             self.colors['text']),
            ("BPM Range", f"±{int(bpm_range)}" if bpm_range else "---", 
             self.colors['text_dim']),
        ]
        
        for i, (label, value, color) in enumerate(metrics):
            col = i % 2
            row = i // 2
            
            x = panel_x + 50 + col * 450
            y = panel_y + 60 + row * 90
            
            cv2.putText(canvas, label, (x, y), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, self.colors['text_dim'], 1)
            cv2.putText(canvas, value, (x, y+40), 
                       cv2.FONT_HERSHEY_BOLD, 1.2, color, 2)
        
        return canvas
    
    def draw_graph_panel(self, canvas):
        """Draw grafik BPM dan SQI history."""
        # Panel position (kanan bawah)
        panel_x = 980
        panel_y = 640
        panel_w = 900
        panel_h = 280
        
        # Background
        cv2.rectangle(canvas, (panel_x, panel_y), 
                     (panel_x+panel_w, panel_y+panel_h), self.colors['panel'], -1)
        
        # Title
        cv2.putText(canvas, "REAL-TIME GRAPHS", (panel_x+30, panel_y+35), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, self.colors['text_dim'], 2)
        
        # Graph area
        graph_x = panel_x + 50
        graph_y = panel_y + 60
        graph_w = panel_w - 100
        graph_h = 180
        
        # BPM Graph (top half)
        if len(self.bpm_history) > 1:
            self._draw_line_graph(canvas, graph_x, graph_y, graph_w, graph_h//2, 
                                 list(self.bpm_history), "BPM", 
                                 self.colors['accent'], y_min=50, y_max=120)
        
        # SQI Graph (bottom half)
        if len(self.sqi_history) > 1:
            self._draw_line_graph(canvas, graph_x, graph_y + graph_h//2 + 20, 
                                 graph_w, graph_h//2 - 20, 
                                 list(self.sqi_history), "Quality", 
                                 self.colors['success'], y_min=0, y_max=1)
        
        return canvas
    
    def _draw_line_graph(self, canvas, x, y, w, h, data, label, color, y_min=0, y_max=100):
        """Helper untuk draw line graph."""
        # Border
        cv2.rectangle(canvas, (x, y), (x+w, y+h), (60, 60, 60), 1)
        
        # Label
        cv2.putText(canvas, label, (x+5, y+15), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, self.colors['text_dim'], 1)
        
        # Normalize data
        if len(data) < 2:
            return
        
        points = []
        step = w / max(len(data)-1, 1)
        
        for i, value in enumerate(data):
            px = int(x + i * step)
            # Normalize ke range graph
            normalized = (value - y_min) / (y_max - y_min)
            normalized = max(0, min(1, normalized))  # Clamp
            py = int(y + h - (normalized * h))
            points.append((px, py))
        
        # Draw lines
        for i in range(len(points)-1):
            cv2.line(canvas, points[i], points[i+1], color, 2, cv2.LINE_AA)
        
        # Draw last value
        if data:
            last_val = data[-1]
            val_text = f"{last_val:.1f}" if isinstance(last_val, float) else f"{last_val}"
            cv2.putText(canvas, val_text, (x+w-60, y+15), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
    
    def draw_instructions(self, canvas):
        """Draw instruksi overlay untuk pengguna baru."""
        if not self.show_instructions:
            return canvas
        
        # Semi-transparent overlay
        overlay = canvas.copy()
        cv2.rectangle(overlay, (0, 0), (self.width, self.height), (0, 0, 0), -1)
        alpha = min(0.7, self.instructions_alpha)
        canvas = cv2.addWeighted(overlay, alpha, canvas, 1-alpha, 0)
        
        # Instruction panel
        panel_w = 800
        panel_h = 600
        panel_x = (self.width - panel_w) // 2
        panel_y = (self.height - panel_h) // 2
        
        cv2.rectangle(canvas, (panel_x, panel_y), (panel_x+panel_w, panel_y+panel_h), 
                     self.colors['panel'], -1)
        cv2.rectangle(canvas, (panel_x, panel_y), (panel_x+panel_w, panel_y+panel_h), 
                     self.colors['accent'], 3)
        
        # Title
        cv2.putText(canvas, "How to Use", (panel_x+50, panel_y+70), 
                   cv2.FONT_HERSHEY_BOLD, 1.5, self.colors['accent'], 3)
        
        # Instructions
        instructions = [
            ("1. Position Your Face", "Center your face in the camera view"),
            ("2. Stay Still", "Minimize movement for accurate readings"),
            ("3. Good Lighting", "Ensure your face is well-lit"),
            ("4. Wait 10 Seconds", "Allow system to stabilize"),
            ("5. Check Confidence", "Higher confidence = more reliable"),
            ("", ""),
            ("Press SPACE to hide this guide", ""),
            ("Press Q to quit", ""),
        ]
        
        y_offset = panel_y + 130
        for title, desc in instructions:
            if title:
                cv2.putText(canvas, title, (panel_x+80, y_offset), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.9, self.colors['text'], 2)
                if desc:
                    cv2.putText(canvas, desc, (panel_x+100, y_offset+30), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.6, self.colors['text_dim'], 1)
                y_offset += 70
            else:
                y_offset += 20
        
        return canvas
    
    def draw_footer(self, canvas):
        """Draw footer dengan controls."""
        # Footer bar
        y_start = self.height - 50
        cv2.rectangle(canvas, (0, y_start), (self.width, self.height), 
                     self.colors['panel'], -1)
        
        # Controls
        controls = "SPACE: Toggle Instructions | Q: Quit | ESC: Exit Fullscreen"
        cv2.putText(canvas, controls, (50, y_start+35), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, self.colors['text_dim'], 1)
        
        return canvas
    
    def update(self, video_frame, bpm=None, confidence=None, sqi=None, 
               motion_detected=False, roi_quality=None):
        """Update UI dengan data terbaru."""
        # Create canvas
        canvas = self.create_canvas()
        
        # Calculate FPS
        current_time = time.time()
        fps_value = 1.0 / (current_time - self.start_time) if hasattr(self, 'last_update_time') else 30
        self.last_update_time = current_time
        
        # Draw all components
        canvas = self.draw_header(canvas, fps_value)
        canvas = self.draw_video_panel(canvas, video_frame, roi_quality)
        canvas = self.draw_bpm_display(canvas, bpm, confidence)
        
        # Calculate metrics
        avg_bpm = np.mean(list(self.bpm_history)[-30:]) if len(self.bpm_history) >= 10 else None
        bpm_range = np.std(list(self.bpm_history)[-30:]) if len(self.bpm_history) >= 10 else None
        
        canvas = self.draw_metrics_panel(canvas, sqi, motion_detected, avg_bpm, bpm_range)
        canvas = self.draw_graph_panel(canvas)
        canvas = self.draw_footer(canvas)
        
        # Instructions overlay
        if self.show_instructions:
            canvas = self.draw_instructions(canvas)
        
        # Update histories
        if bpm is not None:
            self.bpm_history.append(bpm)
        if sqi is not None:
            self.sqi_history.append(sqi)
        if confidence is not None:
            self.confidence_history.append(confidence)
        
        return canvas

print("✅ FullscreenVisualizer created - Modern UI untuk 1920x1080")

## 🎯 Fullscreen rPPG System dengan UI Modern

In [ ]:
class FullscreenRPPGSystem:
    """
    Complete rPPG system dengan modern fullscreen UI.
    Mengintegrasikan semua komponen: detection, processing, dan visualization.
    """
    
    def __init__(self):
        print("Initializing Fullscreen rPPG System...")
        
        # Initialize components
        self.face_detector = FaceDetector()
        self.roi_selector = AdaptiveROISelector()
        self.signal_processor = AdvancedSignalProcessor()
        self.visualizer = FullscreenVisualizer()
        
        # Signal buffer
        self.signal_buffer = deque(maxlen=300)  # 10 detik di 30 FPS
        self.timestamp_buffer = deque(maxlen=300)
        
        # Frame quality tracking
        self.quality_buffer = deque(maxlen=100)
        
        # State
        self.running = False
        self.bpm = None
        self.confidence = None
        self.sqi = None
        self.motion_detected = False
        self.roi_quality = None
        
        # Statistics
        self.total_frames = 0
        self.successful_frames = 0
        
        print("✅ System initialized successfully!")
        print("\n📌 Instructions:")
        print("   - Position your face in the camera")
        print("   - Ensure good lighting conditions")
        print("   - Stay still for accurate measurements")
        print("   - SPACE: Toggle instructions overlay")
        print("   - Q or ESC: Quit application")
        print("\nPress any key to start...")
    
    def process_frame(self, frame):
        """Process single frame dan return visualization."""
        self.total_frames += 1
        
        # Face detection
        face_mesh = self.face_detector.detect(frame)
        
        if face_mesh is None:
            # No face detected
            canvas = self.visualizer.update(
                video_frame=frame,
                bpm=None,
                confidence=None,
                sqi=None,
                motion_detected=False,
                roi_quality=None
            )
            return canvas
        
        # Adaptive ROI selection
        forehead_roi, left_cheek_roi, right_cheek_roi = self.face_detector.extract_roi(frame, face_mesh)
        
        if forehead_roi is None:
            canvas = self.visualizer.update(
                video_frame=frame,
                bpm=None,
                confidence=None,
                sqi=None,
                motion_detected=False,
                roi_quality=None
            )
            return canvas
        
        # Pilih ROI terbaik
        best_roi, self.roi_quality = self.roi_selector.select_best_roi(
            forehead_roi, left_cheek_roi, right_cheek_roi
        )
        
        # Draw ROI pada video frame untuk feedback
        frame_with_roi = frame.copy()
        if best_roi is not None:
            # Highlight selected ROI
            mask = np.zeros_like(frame_with_roi)
            mask[best_roi > 0] = (0, 255, 0)
            frame_with_roi = cv2.addWeighted(frame_with_roi, 0.9, mask, 0.1, 0)
        
        # Extract signal
        current_time = time.time()
        rgb_signal = self.signal_processor.extract_signal(best_roi)
        
        if rgb_signal is not None:
            self.successful_frames += 1
            self.signal_buffer.append(rgb_signal)
            self.timestamp_buffer.append(current_time)
            
            # Process jika buffer cukup
            if len(self.signal_buffer) >= 90:  # Minimal 3 detik
                # Convert buffer ke array
                signal_array = np.array(list(self.signal_buffer))
                timestamps = np.array(list(self.timestamp_buffer))
                
                # Estimate BPM dengan advanced processing
                result = self.signal_processor.estimate_bpm_advanced(
                    signal_array, 
                    timestamps,
                    previous_bpm=self.bpm
                )
                
                if result is not None:
                    self.bpm = result['bpm']
                    self.confidence = result['confidence']
                    self.sqi = result['sqi']
                    self.motion_detected = result['motion_detected']
        
        # Update visualization
        canvas = self.visualizer.update(
            video_frame=frame_with_roi,
            bpm=self.bpm,
            confidence=self.confidence,
            sqi=self.sqi,
            motion_detected=self.motion_detected,
            roi_quality=self.roi_quality
        )
        
        return canvas
    
    def start(self):
        """Start real-time system dengan fullscreen UI."""
        self.running = True
        cap = cv2.VideoCapture(0)
        
        if not cap.isOpened():
            print("❌ Error: Cannot access camera!")
            return
        
        # Set camera resolution
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
        cap.set(cv2.CAP_PROP_FPS, 30)
        
        # Create fullscreen window
        window_name = "rPPG Heart Rate Monitor - Fullscreen"
        cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
        cv2.setWindowProperty(window_name, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
        
        print("\n🚀 System started!")
        print("Camera initialized - Processing frames...")
        
        frame_count = 0
        start_time = time.time()
        
        try:
            while self.running:
                ret, frame = cap.read()
                
                if not ret:
                    print("⚠ Warning: Failed to grab frame")
                    continue
                
                frame_count += 1
                
                # Process frame
                canvas = self.process_frame(frame)
                
                # Display
                cv2.imshow(window_name, canvas)
                
                # Handle keyboard input
                key = cv2.waitKey(1) & 0xFF
                
                if key == ord('q') or key == 27:  # Q or ESC
                    print("\n🛑 Stopping system...")
                    break
                elif key == ord(' '):  # SPACE
                    self.visualizer.show_instructions = not self.visualizer.show_instructions
                    if not self.visualizer.show_instructions:
                        print("ℹ Instructions hidden")
                    else:
                        print("ℹ Instructions shown")
                elif key == ord('f'):  # F untuk toggle fullscreen
                    is_fullscreen = cv2.getWindowProperty(window_name, cv2.WND_PROP_FULLSCREEN)
                    if is_fullscreen == cv2.WINDOW_FULLSCREEN:
                        cv2.setWindowProperty(window_name, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_NORMAL)
                        print("ℹ Windowed mode")
                    else:
                        cv2.setWindowProperty(window_name, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
                        print("ℹ Fullscreen mode")
        
        except KeyboardInterrupt:
            print("\n⚠ Interrupted by user")
        
        finally:
            # Cleanup
            self.running = False
            cap.release()
            cv2.destroyAllWindows()
            
            # Statistics
            elapsed_time = time.time() - start_time
            avg_fps = frame_count / elapsed_time if elapsed_time > 0 else 0
            success_rate = (self.successful_frames / self.total_frames * 100) if self.total_frames > 0 else 0
            
            print("\n" + "="*60)
            print("📊 SESSION SUMMARY")
            print("="*60)
            print(f"Total Runtime: {elapsed_time:.1f} seconds")
            print(f"Frames Processed: {self.total_frames}")
            print(f"Average FPS: {avg_fps:.1f}")
            print(f"Successful Detections: {self.successful_frames} ({success_rate:.1f}%)")
            
            if self.bpm is not None and self.confidence is not None:
                print(f"\nFinal Heart Rate: {self.bpm:.1f} BPM")
                print(f"Confidence: {self.confidence*100:.1f}%")
                
                if self.confidence > 0.7:
                    reliability = "RELIABLE ✓"
                elif self.confidence > 0.5:
                    reliability = "MODERATE"
                else:
                    reliability = "LOW - Take another measurement"
                
                print(f"Reliability: {reliability}")
            
            print("="*60)
            print("✅ System stopped successfully")

print("✅ FullscreenRPPGSystem created - Ready to run!")

## ▶️ Jalankan Fullscreen System

Eksekusi cell berikut untuk menjalankan rPPG dengan UI fullscreen modern:

**Keyboard Controls:**
- **SPACE** - Toggle panduan instruksi
- **F** - Toggle fullscreen/windowed mode  
- **Q atau ESC** - Keluar dari aplikasi

**Tips untuk hasil terbaik:**
1. Pastikan wajah terlihat jelas di kamera
2. Gunakan pencahayaan yang baik (tidak terlalu gelap/terang)
3. Tetap diam selama pengukuran (minimal 10 detik)
4. Perhatikan indikator confidence - makin tinggi makin akurat

In [ ]:
# Jalankan Fullscreen rPPG System
if __name__ == "__main__":
    system = FullscreenRPPGSystem()
    system.start()